In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import cv2
import numpy as np

In [2]:
batch_size = 32
img_height = 100
img_width = 100

In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/Users/danielkashkett/Desktop/EyeProject/data/eye_imgs/',
  validation_split=0.2,
    color_mode="grayscale",
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/Users/danielkashkett/Desktop/EyeProject/data/eye_imgs/',
  validation_split=0.2,
  color_mode="grayscale",
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 20151 files belonging to 5 classes.
Using 16121 files for training.
Found 20151 files belonging to 5 classes.
Using 4030 files for validation.


In [4]:
class_names = train_ds.class_names
print(class_names)

['center', 'down', 'left', 'right', 'up']


In [5]:
def create_model(): 
    num_classes = 5

    model = Sequential([
      layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 1)),
      layers.Conv2D(16, 1, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(32, 1, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(64, 1, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Dropout(0.2),
      layers.Flatten(),
      layers.Dense(128, activation='relu'),
      layers.Dense(num_classes)
    ])
    opt = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
    
    model.compile(optimizer="adam", 
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

In [6]:
model = create_model()
epochs = 5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/5
504/504 [==============================] - 35s 70ms/step - loss: 0.8329 - accuracy: 0.6954 - val_loss: 0.4422 - val_accuracy: 0.8600
Epoch 2/5
504/504 [==============================] - 34s 68ms/step - loss: 0.3898 - accuracy: 0.8764 - val_loss: 0.3317 - val_accuracy: 0.9025
Epoch 3/5
504/504 [==============================] - 36s 71ms/step - loss: 0.2904 - accuracy: 0.9059 - val_loss: 0.2822 - val_accuracy: 0.9062
Epoch 4/5
504/504 [==============================] - 35s 70ms/step - loss: 0.2435 - accuracy: 0.9189 - val_loss: 0.2367 - val_accuracy: 0.9191
Epoch 5/5
504/504 [==============================] - 36s 72ms/step - loss: 0.2114 - accuracy: 0.9265 - val_loss: 0.2289 - val_accuracy: 0.9223


In [7]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/5
504/504 [==============================] - 32s 63ms/step - loss: 0.1904 - accuracy: 0.9337 - val_loss: 0.2094 - val_accuracy: 0.9288
Epoch 2/5
504/504 [==============================] - 36s 72ms/step - loss: 0.1746 - accuracy: 0.9378 - val_loss: 0.2042 - val_accuracy: 0.9305
Epoch 3/5
504/504 [==============================] - 34s 67ms/step - loss: 0.1595 - accuracy: 0.9440 - val_loss: 0.1998 - val_accuracy: 0.9330
Epoch 4/5
504/504 [==============================] - 35s 69ms/step - loss: 0.1541 - accuracy: 0.9448 - val_loss: 0.2024 - val_accuracy: 0.9355
Epoch 5/5
504/504 [==============================] - 34s 68ms/step - loss: 0.1450 - accuracy: 0.9482 - val_loss: 0.1795 - val_accuracy: 0.9402


In [8]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=2
)

Epoch 1/2
504/504 [==============================] - 32s 63ms/step - loss: 0.1363 - accuracy: 0.9506 - val_loss: 0.1895 - val_accuracy: 0.9380
Epoch 2/2
504/504 [==============================] - 36s 71ms/step - loss: 0.1280 - accuracy: 0.9548 - val_loss: 0.1776 - val_accuracy: 0.9402


In [9]:
model.save('./Models/jupiter1')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./Models/jupiter1/assets


In [2]:
model = tf.keras.models.load_model('./Models/jupiter1')

In [55]:
img1 = cv2.imread('../data/eye_imgs/right/a1.jpg' ,cv2.IMREAD_UNCHANGED)
img2 = cv2.imread('../data/eye_imgs/left/a10.jpg' ,cv2.IMREAD_UNCHANGED)
# img = img.reshape((100,100,1))
# img = np.expand_dims(img,0)

In [56]:
predict_both(left = img1, right = img2)

3

In [24]:
def joint_prediction():
    pass

In [42]:
img1 = np.expand_dims(img1,0)
img2 = np.expand_dims(img2,0)

In [43]:
a = np.concatenate((img1,img2))

In [27]:
a.shape

(2, 100, 100)

In [45]:
model.predict(a)

array([ 2.0141094, -0.907617 ,  3.9984462, -2.9328032,  1.9947188],
      dtype=float32)

In [53]:
def predict_both(left, right):
    img1 = np.expand_dims(left,0)
    img2 = np.expand_dims(right,0)
    concat = np.concatenate((img1,img2))
    classes = model.predict_classes(concat)
    if classes[0] == classes[1]:
        return classes[0]
    else:
        pred = model.predict(concat)
        a = (pred[0].argmax(), pred[0].max())
        b = (pred[1].argmax(), pred[1].max())
        if a[1] > b[1]:
            return a[0]
        else:
            return b[0]

In [ ]:
def predict_frame(self, eye_img):
    img = eye_img.copy()
    batch = np.expand_dims(img,0)
    prediction = self.model.predict_classes(batch)[0]
    return prediction

In [211]:
class_names

['center', 'down', 'left', 'right', 'up']

In [212]:
model.predict_classes(img)[0]

2